In [1]:
import requests
from bs4 import BeautifulSoup 

In [2]:
class ArticleCrawling:

    def __init__(self, url):
        self.url = url
        self.soup = self._get_soup()

    def _get_soup(self):
        response = requests.get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    def get_article_url(self):
        meta_tag = self.soup.find("meta", property = "og:url")
        return meta_tag["content"] if meta_tag else None
    
    def get_id(self):
        url = self.get_article_url()    # https://www.spnews.co.kr/news/articleView.html?idxno=101049

        # source(spnew, ytn 등등)
        source = url.split("//")[1]
        source = source.split("/")[0]
        source = source.replace("www.", "").split(".")[0]

        # id
        if "idxno" not in url:
            return None
        idx = url.split("idxno=")[1]
        return f"{source}_{idx}"
    
    def get_title(self):
        header = self.soup.find("h1", class_ = "heading")
        return header.get_text(strip = True) if header else None
      
    def get_author(self):
        tag = self.soup.find("i", class_ = "icon-user-o")
        if tag:
            author = tag.parent.get_text(strip = True)
            return author.replace("기자명", "").strip()
        
    def get_category(self):
        breadcrumb = self.soup.find("ul", class_ = "breadcrumbs")
        if breadcrumb:
            for a in breadcrumb.find_all("a"):
                text = a.get_text(strip=True)
                if text in ["정치", "외교", "군사", "경제/산업", "사회/문화/체육"]:
                    return text
    
    def get_publish_date(self):
        tag = self.soup.find("i", class_ = "icon-clock-o")
        if tag:
            raw = tag.parent.get_text(strip = True)
            return raw.replace("입력", "").strip()
        
    def get_contents(self):
        article_body = self.soup.find_all("span", style = "font-size:18px;")
        contents = "\n".join(i.get_text(strip = True) for i in article_body)
        # return contents.rstrip("@") 
        return contents.split('@')[0].strip()
    
    def to_dict(self):
        return {
            "url": self.get_article_url(),
            "id": self.get_id(),
            "title": self.get_title(),
            "author": self.get_author(),
            "category": self.get_category(),
            "publish_date": self.get_publish_date(),
            "contents": self.get_contents()
        }
    
    def __str__(self):
        data = self.to_dict()
        return data


In [3]:
import pandas as pd

In [4]:
url_list = [
    'https://www.spnews.co.kr/news/articleView.html?idxno=101049',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101366',
    'https://www.spnews.co.kr/news/articleView.html?idxno=100345',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101306',
    'https://www.spnews.co.kr/news/articleView.html?idxno=100889',
    'https://www.spnews.co.kr/news/articleView.html?idxno=100318',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101377',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101097',
    'https://www.spnews.co.kr/news/articleView.html?idxno=100356',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101404',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101394',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101258',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101402',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101369',
    'https://www.spnews.co.kr/news/articleView.html?idxno=101272'
]

In [5]:
def make_test_dataset(url_list):
    test_df = pd.DataFrame(columns=['id', 'title', 'contents', 'author', 'publish_date', 'url', 'category'])

    for url in url_list:
        test1 = ArticleCrawling(url).to_dict()

        # DataFrame에 한 행 추가
        test_df.loc[len(test_df)] = [
            test1['id'],
            test1['title'],
            test1['contents'],
            test1['author'],
            test1['publish_date'],
            test1['url'],
            test1['category'],
        ]

    return test_df.reset_index(drop=True)


In [6]:
test_df = make_test_dataset(url_list)

In [7]:
test_df.head(5)

,id,title,contents,author,publish_date,url,category
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025.11.06 07:40,https://www.spnews.co.kr/news/articleView.html...,정치
1,spnews_101366,"박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다.\n노동신...,안윤석 대기자,2025.11.16 06:47,https://www.spnews.co.kr/news/articleView.html...,정치
2,spnews_100345,"김정은, '신의주온실종합농장' 또 현지지도...""농촌문명의 새로운 경지 개척""",김정은 북한 총비서가 17일 마감단계에 들어선 신의주온실종합농장 건설장을 또다시 방...,안윤석 대기자,2025.10.18 07:02,https://www.spnews.co.kr/news/articleView.html...,정치
3,spnews_101306,"박태성 北 내각총리, 통싸완 폼비한 라오스 외무상 접견",박태성 북한 내각총리가 13일 만수대의사당에서 통싸완 폼비한 라오스 외무상을 만났다...,안윤석 대기자,2025.11.14 07:17,https://www.spnews.co.kr/news/articleView.html...,외교
4,spnews_100889,"북한-러시아, 무역경제 및 과학기술협력 추진 회담...의정서 조인",북한과 러시아가 합의된 다방면적인 쌍무협력계획 이행을 추진하기 위한 회담을 31일 ...,안윤석 대기자,2025.11.01 07:15,https://www.spnews.co.kr/news/articleView.html...,외교


# LLM Summarizer 구현

In [14]:
test_data = test_df.copy()

In [15]:
test_data.head(2)

,id,title,contents,author,publish_date,url,category
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025.11.06 07:40,https://www.spnews.co.kr/news/articleView.html...,정치
1,spnews_101366,"박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다.\n노동신...,안윤석 대기자,2025.11.16 06:47,https://www.spnews.co.kr/news/articleView.html...,정치


In [17]:
test_data['publish_date'] = pd.to_datetime(test_data['publish_date']).dt.date
test_data.head(2)

,id,title,contents,author,publish_date,url,category
0,spnews_101049,"北, 김영남 상임위원장 영결식...김정은 참석",고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로...,안윤석 대기자,2025-11-06,https://www.spnews.co.kr/news/articleView.html...,정치
1,spnews_101366,"박태성 北 내각총리, 광산·발전소 등 경제 여러 부문 현장 확인",박태성 북한 내각총리가 광산과 발전소 등 경제 여러 부문 현장을 확인했다.\n노동신...,안윤석 대기자,2025-11-16,https://www.spnews.co.kr/news/articleView.html...,정치


In [ ]:
from openai import OpenAI
import pandas as pd

client = OpenAI(api_key="YOUR_API_KEY")  # 반드시 환경변수로 관리 권장

# test_data는 기존 test_df.copy() 로 생성된 df라고 가정
df = test_data.copy()

# 새 컬럼 추가
new_cols = ["summary", "keywords", "event_title", "event_date", "event_obj", "event_loc"]
for col in new_cols:
    df[col] = None


def generate_event_info(title, contents, publish_date):
    """뉴스 기사 정보를 LLM으로 요약하고 항목별 데이터 반환"""
    prompt = f"""
아래 기사를 분석하여 요구된 정보를 작성하시오.

기사 제목:
{title}

기사 내용:
{contents}

기사 작성일:
{publish_date}

1. 아래 형식으로 정리
- 주요 사건 요약(summary):
- 사건 주제(event_title):
- 사건 발생일(event_date):
- 사건 발생 주체 기관(event_obj):
- 사건 발생 지명(event_loc):
- 주요 키워드(keywords):

2. 각 카테고리의 조건
- "주요 사건 요약"은 3문장 이하로 핵심 내용만 발췌
- "사건 발생일" 과 "기사 작성일"은 yyyy-mm-dd 형식
  명시되지 않으면 작성일 기준 계산
- "사건 발생 주체 기관"은 기관명만, 다수면 쉼표로 구분
- "사건 발생 지명"은 하나만 선택
- "주요 키워드"는 5개
- 출력은 딕셔너리 형태로, 값이 여러 개면 리스트로 묶어서 출력
- 설명 금지, 답만 출력
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 북한 관련 뉴스 사건 정보를 추출하는 전문 분석 모델입니다."},
            {"role": "user", "content": prompt},
        ],
        temperature=0
    )

    result_text = response.choices[0].message.content.strip()

    # 문자열을 dict로 변환
    try:
        result = eval(result_text)  # dict로 파싱 (구조가 명확하므로 안전)
    except:
        print("Parsing error:", result_text)
        return None
    
    return result



# 각 row 반복 → LLM 호출 → 결과 입력
for idx, row in df.iterrows():
    print(f"Processing index {idx} ...")

    title = row["title"]
    contents = row["contents"]
    publish_date = row["publish_date"]  # datetime.date

    result = generate_event_info(title, contents, publish_date)

    if result:
        df.at[idx, "summary"] = result.get("summary")
        df.at[idx, "keywords"] = result.get("keywords")
        df.at[idx, "event_title"] = result.get("event_title")
        df.at[idx, "event_date"] = result.get("event_date")
        df.at[idx, "event_obj"] = result.get("event_obj")
        df.at[idx, "event_loc"] = result.get("event_loc")

print("작업 완료!")